In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
from recommender_tool import FashionOutfitGenerator
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.chains import ConversationalRetrievalChain,LLMChain
from langchain.chains.question_answering import load_qa_chain
from config import *
from langchain.vectorstores.redis import Redis
from langchain.embeddings.openai import OpenAIEmbeddings
import requests

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY1

In [2]:
REDIS_URL = 'redis://localhost:6380'

In [3]:
search_terms = [
    'lehenga',
    'saree',
    't-shirt',
    'shirt',
    'jeans',
    'shorts',
    'crop-tops',
    'jackets',
    'sweatshirts',
    'plazzo',
    'salwar',
    'churidar',
    'dhoti',
    'jumpsuit',
    'blazer',
    'coat',
    'sweater',
    'shrug',
]

In [4]:
def search_flipkart( search_term):
    URL = f"https://9d5f-13-233-172-64.ngrok-free.app/search/{search_term}"
    response = requests.get(URL).json()
    responses = response["result"]
    return responses

In [9]:
texts = []
metadatas = []
for search_term in search_terms:
    results = search_flipkart(search_term)
    texts.extend([result["name"] for result in results])
    metadatas.extend(results)

In [10]:
print(results[0])

{'name': 'Women Half Sleeve Blue Shrug', 'link': 'https://www.flipkart.com/raagvisha-women-shrug/p/itm3d9277448e515', 'current_price': 742, 'original_price': 1999, 'discounted': True, 'thumbnail': [], 'query_url': 'https://flipkart.com/product/raagvisha-women-shrug/p/itm3d9277448e515'}


In [6]:
embeddings = OpenAIEmbeddings()
vectorstore = Redis.from_existing_index(
    index_name=INDEX_NAME,
    embedding=embeddings,
    redis_url="redis://@localhost:6380",
)

In [7]:
vectorstore.search("cotton shirt",search_type="similarity")

[Document(page_content='Digital Print Semi Stitched Lehenga &amp; Crop Top', metadata={'uuid': '4a65bc1a-618e-4e5f-9e9b-152c0a9390e8', 'id': 'doc:database-index:9788892b90024b0cb253df172865191c'}),
 Document(page_content='Embroidered Semi Stitched Lehenga &amp; Crop Top', metadata={'uuid': 'f5c13e78-3116-4d09-904b-bf00eb75ad59', 'id': 'doc:database-index:cc126c46c7a8417d89d3663f92a00205'}),
 Document(page_content='Printed Semi Stitched Lehenga Choli', metadata={'uuid': '75801468-9c6a-473d-b35c-43a4819299b7', 'id': 'doc:database-index:ca8c389557814e1f8f8cc07d6387e1ec'}),
 Document(page_content='Printed Semi Stitched Lehenga Choli', metadata={'uuid': '6dea472c-2e34-4ed7-ba9c-14f39e1adc36', 'id': 'doc:database-index:3878cb6512e846ff8cf216dd5bbd9212'})]

In [148]:
memory = VectorStoreRetrieverMemory(retriever=retriever,memory_key="history")

In [149]:
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative 
and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=_DEFAULT_TEMPLATE
)

In [150]:
llm = ChatOpenAI(temperature=0)
llmchain = ConversationChain(llm=llm,memory=memory,verbose=True,prompt=PROMPT)

In [151]:
llmchain.predict(input="Hi, Suggest me an outfit for a party.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: Hi
response: Hello! How can I assist you today?
input: What is my age ?
response: Your age is 12.
input: What is my age ?
response: Your age is 12.
input: What is my age ?
response: Your age is 12.

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Hi, Suggest me an outfit for a party.
AI:

> Finished chain.


"Hello! I'd be happy to help you find an outfit for a party. Could you please provide me with some more information? What is the dress code for the party? Is there a specific theme or color scheme?"

In [153]:
llmchain.predict(input="The party is a summer swimming pool party.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: Hi, Suggest me an outfit for a party.
response: Hello! I'd be happy to help you find an outfit for a party. Could you please provide me with some more information? What is the dress code for the party? Is there a specific theme or color scheme?
input: What is my age ?
response: Your age is 12.
input: What is my age ?
response: Your age is 12.
input: What is my age ?
response: Your age is 12.

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: The party is a summer swimming pool party.
AI:

> Finished chain.


"That sounds like a fun event! For a summer swimming pool party, you'll want to wear something comfortable and appropriate for the water. A popular choice is a swimsuit or bikini paired with a stylish cover-up or sarong. You can also consider wearing a lightweight sundress or a tank top with shorts. Don't forget to bring a towel, sunscreen, and a pair of flip-flops or sandals for the poolside. Enjoy the party and have a great time!"

In [154]:
llmchain.predict(input="Suggest something else instead of tank top.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: The party is a summer swimming pool party.
response: That sounds like a fun event! For a summer swimming pool party, you'll want to wear something comfortable and appropriate for the water. A popular choice is a swimsuit or bikini paired with a stylish cover-up or sarong. You can also consider wearing a lightweight sundress or a tank top with shorts. Don't forget to bring a towel, sunscreen, and a pair of flip-flops or sandals for the poolside. Enjoy the party and have a great time!
input: Hi, Suggest me an outfit for a party.
response: Hello! I'd be happy to help you find an outfit for a party. Could you please provide me with some more info

"Sure! Instead of a tank top, you could consider wearing a lightweight blouse or a flowy tunic. These options will still keep you cool and comfortable for the summer swimming pool party. Another alternative could be a crop top paired with high-waisted shorts or a skirt. This will give you a trendy and stylish look while still being appropriate for the occasion. Don't forget to accessorize with some statement jewelry or a cute hat to complete your outfit!"

In [155]:
llmchain.predict(input="Alright, Summarize the final outfit then.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: Hi, Suggest me an outfit for a party.
response: Hello! I'd be happy to help you find an outfit for a party. Could you please provide me with some more information? What is the dress code for the party? Is there a specific theme or color scheme?
input: The party is a summer swimming pool party.
response: That sounds like a fun event! For a summer swimming pool party, you'll want to wear something comfortable and appropriate for the water. A popular choice is a swimsuit or bikini paired with a stylish cover-up or sarong. You can also consider wearing a lightweight sundress or a tank top with shorts. Don't forget to bring a towel, sunscreen, and

"Certainly! For your summer swimming pool party, the final outfit could be a stylish swimsuit or bikini paired with a lightweight blouse or flowy tunic. This will keep you cool and comfortable while still being appropriate for the occasion. You can also consider wearing high-waisted shorts or a skirt with a crop top for a trendy and stylish look. Don't forget to accessorize with statement jewelry or a cute hat to complete your outfit. And remember to bring a towel, sunscreen, and a pair of flip-flops or sandals for the poolside. Have a fantastic time at the party!"

In [157]:
llmchain.predict(input="Suggest me the earlier option instead of the blouse.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: Suggest something else instead of tank top.
response: Sure! Instead of a tank top, you could consider wearing a lightweight blouse or a flowy tunic. These options will still keep you cool and comfortable for the summer swimming pool party. Another alternative could be a crop top paired with high-waisted shorts or a skirt. This will give you a trendy and stylish look while still being appropriate for the occasion. Don't forget to accessorize with some statement jewelry or a cute hat to complete your outfit!
input: Hi, Suggest me an outfit for a party.
response: Hello! I'd be happy to help you find an outfit for a party. Could you please provid

"Sure! Instead of the blouse, you could consider wearing a lightweight tunic or a flowy dress. These options will still keep you cool and comfortable for the summer swimming pool party. Another alternative could be a crop top paired with high-waisted shorts or a skirt. This will give you a trendy and stylish look while still being appropriate for the occasion. Don't forget to accessorize with some statement jewelry or a cute hat to complete your outfit!"